In [267]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte


In [268]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import copy


In [269]:

data = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv')
data_test = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')

data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [270]:
data_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data Preprocessing

**Checking null value**

In [271]:
print(data.isnull().sum())

label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64


**Splitting and separating data**

Splitting data into train and val set with a ratio of 0.8

In [272]:
# Splitting data
target_column = 'label' 
shuffled_indices = np.random.permutation(len(data))
data_shuffled = data.iloc[shuffled_indices]

# Define the split ratio
split_ratio = 0.8
split_index = int(len(data) * split_ratio)

# Split the data
train_data = data_shuffled.iloc[:split_index]
val_data = data_shuffled.iloc[split_index:]

# Separate features and target variable
x_train = train_data.drop(target_column, axis=1)
y_train = train_data[target_column].values.reshape(-1, 1)  # Reshape for consistency
x_val = val_data.drop(target_column, axis=1)
y_val = val_data[target_column].values.reshape(-1, 1)  # Reshape for consistency

In [273]:
y_val.shape

(2000, 1)

**Separating test set**

In [274]:
shuffled_indices = np.random.permutation(len(data_test))
data_shuffled_test = data_test.iloc[shuffled_indices]

# Separate features and target variable
x_test = data_shuffled_test.drop(target_column, axis=1)
y_test = data_shuffled_test[target_column].values.reshape(-1, 1)  # Reshape for consistency


**Checking shape**

In [275]:
print(f"Training features shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation features shape: {x_val.shape}")
print(f"Validation labels shape: {y_val.shape}")

Training features shape: (8000, 784)
Training labels shape: (8000, 1)
Validation features shape: (2000, 784)
Validation labels shape: (2000, 1)


In [276]:
# Example: Labels (8000 examples, 1 class per example)
y_train = pd.Series([0, 1, 2, 3, 4, 5, 6, 7, 8, 9] * 800)

# One-hot encode using pandas
y_train = pd.get_dummies(y_train).values.T  # Shape: (10, 8000)

In [277]:
x_train = x_train.T
x_val = x_val.T
print("x_train shape after transpose:", x_train.shape)  # (784, 8000)
print("x_val shape after transpose:", x_val.shape)      # (784, 2000)

x_train shape after transpose: (784, 8000)
x_val shape after transpose: (784, 2000)


**Visualising the data:**

Standardising the data 

In [278]:
x_train = x_train/255
y_train = y_train/255
x_val = x_train/255
y_val = y_val/255
x_test = x_test/255
y_test = y_test/255
#print(y_test.shape)

```python
def initialize_parameters(n_x, n_h, n_y):
    ...
    return parameters 
def linear_activation_forward(A_prev, W, b, activation):
    ...
    return A, cache
def compute_cost(AL, Y):
    ...
    return cost
def linear_activation_backward(dA, cache, activation):
    ...
    return dA_prev, dW, db
def update_parameters(parameters, grads, learning_rate):
    ...
    return parameters
```

    Argument:
    n_in -- size of the input layer
    n_hid -- size of the hidden layer
    n_out -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_hid, n_in)
                    b1 -- bias vector of shape (n_hid, 1)
                    W2 -- weight matrix of shape (n_out, n_hid)
                    b2 -- bias vector of shape (n_out, 1)

In [279]:
def initialize_parameter(n_in, n_hid, n_out):
    W1 = np.random.randn(n_hid, n_in)*0.01
    b1 = np.zeros((n_hid, 1))
    W2 = np.random.rand(n_out, n_hid)*0.01
    b2 = np.zeros((n_out, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters


    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    

In [280]:
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [281]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) +b
    cache = (A, W, b)
    return Z, cache

In [282]:
def linear_activation_forward(A_prev, W, b, activation):    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z) 
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
        
    cache = (linear_cache, activation_cache)
    return A, cache

In [283]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2 
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, W = parameters[f"W{l}"], b = parameters[f"b{l}"], activation = "relu") 
        caches.append(cache)

    AL, cache = linear_activation_forward(A, W = parameters[f"W{L}"], b= parameters[f"b{L}"], activation = "softmax")
    caches.append(cache)
    
    return AL, caches

In [284]:
'''def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -(1/m)*np.sum(Y* np.log(AL) + (1-Y) * np.log(1-AL))
    cost = np.squeeze(cost)   
    return cost'''
def compute_cost(AL, Y):
    m = Y.shape[1]  # number of examples
    cost = -np.sum(Y * np.log(AL)) / m
    AL = np.clip(AL, 1e-10, 1 - 1e-10)

    return cost

In [285]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis =1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
 
    
    return dA_prev, dW, db

In [286]:
def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache
    
def softmax(Z):
    # Subtracting the max value from Z for numerical stability
    shiftZ = Z - np.max(Z, axis=0, keepdims=True)
    # Calculating the exponentials
    expZ = np.exp(shiftZ)
    # Normalizing to get probabilities
    A = expZ / np.sum(expZ, axis=0, keepdims=True)
    cache = Z
    return A, cache

In [287]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ =relu_backward(dA, activation_cache)
        dA_prev, dW, db =  linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        dZ =sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db =  linear_backward(dZ, linear_cache)
        
    elif activation == "softmax":
        dZ = softmax_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [288]:
def sigmoid_backward(dA, activation_cache):
    Z = activation_cache
    # Sigmoid activation function
    sigmoid_Z = 1 / (1 + np.exp(-Z))  # Sigmoid output
    
    # Derivative of sigmoid
    dZ = dA * sigmoid_Z * (1 - sigmoid_Z)
    
    return dZ

In [289]:
def relu_backward(dA, activation_cache):
    Z = activation_cache  # Z was stored during the forward pass
    dZ = np.array(dA, copy=True)  # Initialize dZ as dA
    
    # When Z <= 0, set dZ to 0 (ReLU derivative)
    dZ[Z <= 0] = 0
    
    return dZ

In [290]:
def softmax_backward(dA, activation_cache):
    Z = activation_cache
    shiftZ = Z-np.max(Z, axis = 0, keepdims=True)
    expZ = np.exp(shiftZ)
    A = expZ / np.sum(expZ, axis = 0, keepdims=True)
     # Compute dZ using the chain rule
    # dZ = dA * Jacobian of softmax
    # Jacobian matrix: diag(A) - A * A^T
    dZ = A * (dA - np.sum(dA * A, axis = 0, keepdims=True))
    return dZ

In [291]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation="softmax")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp

    for l in reversed(range(L-1)):

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [292]:
def update_parameters(params, grads, learning_rate):
 
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2 
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads[f"dW{l+1}"] 
        parameters["b" + str(l+1)] -= learning_rate * grads[f"db{l+1}"]
 
    return parameters

In [293]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
 
    np.random.seed(1)
    costs = []            
    parameters = initialize_parameters_deep(layers_dims)

    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X, parameters)

        cost = compute_cost(AL,Y)
     
        grads = L_model_backward(AL, Y, caches)

        parameters = update_parameters(parameters, grads, learning_rate)
   
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [294]:
n_in = 784     # num_px * num_px * 3
n_hid = 2
n_out = 10
layers_dims = (n_in, n_hid, n_out)
learning_rate = 0.0075

In [295]:
# Train the model and get the parameters and costs
parameters, costs = L_layer_model(x_train, y_train, layers_dims=(n_in, n_hid, n_out), num_iterations=1, print_cost=False)

# Print the cost after the first iteration
print("Cost after first iteration: " + str(costs[0]))



Cost after iteration 0: 0.009029750967739038
Cost after first iteration: 0.009029750967739038


In [296]:
def compute_accuracy(AL, Y):
    """
    AL -- The predicted probabilities (after softmax)
    Y -- The true labels (one-hot encoded)
    """
    predictions = np.argmax(AL, axis=0)  # Convert probabilities to class labels (index of max probability)
    labels = np.argmax(Y, axis=0)  # True labels
    accuracy = np.mean(predictions == labels)  # Compute accuracy
    return accuracy

    
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    


In [297]:
def L_layer_model(X, Y, X_val, Y_val, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):

    np.random.seed(1)
    costs = []                         # keep track of cost
    train_accuracies =[]
    val_accuracies = []
    # Parameters initialization.
    parameters = initialize_parameters_deep(layers_dims)

    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SOFTMAX
        AL, caches = L_model_forward(X, parameters)
        
        # Compute cost.
        cost = compute_cost(AL,Y)

        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
 
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)

        # Compute accuracy for training set
        train_accuracy = compute_accuracy(AL, Y)
        train_accuracies.append(train_accuracy)
        
        # Compute accuracy for validation set
        AL_val, _ = L_model_forward(X_val, parameters)
        val_accuracy = compute_accuracy(AL_val, Y_val)
        val_accuracies.append(val_accuracy)
        
        
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print(f"Iteration {i}: Cost = {cost}, Train Accuracy = {train_accuracy}, Validation Accuracy = {val_accuracy}")
            #print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs, train_accuracies, val_accuracies

In [298]:
print("Cost after first iteration: " + str(costs[0]))

parameters, costs = L_layer_model(x_train, y_train, x_val, y_val, layers_dims, num_iterations = 300, print_cost = True)

Cost after first iteration: 0.009029750967739038
Iteration 0: Cost = 0.009029750967739038, Train Accuracy = 0.099375, Validation Accuracy = 0.02675
Iteration 100: Cost = 0.009029750830355785, Train Accuracy = 0.100375, Validation Accuracy = 0.0
Iteration 200: Cost = 0.009029750695912832, Train Accuracy = 0.100125, Validation Accuracy = 0.0
Iteration 299: Cost = 0.009029750565709135, Train Accuracy = 0.100375, Validation Accuracy = 0.0


ValueError: too many values to unpack (expected 2)

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(12, 5)) 

# Flatten the axes array to iterate through each subplot
axes = axes.flatten()

# Loop through the first 25 images in x_train
for i in range(25):
    image = x_train.iloc[i].values.reshape(28, 28)  # Reshape to 28x28
    label = y_train[i][0]  # Get the label for the image
    
    # Plot the image in the corresponding subplot
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f"Label: {label}", fontsize=8)
    axes[i].axis('off')  # Hide the axis for a cleaner view

plt.tight_layout()
plt.show()